In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from konlpy.tag import Okt
okt = Okt()

In [8]:
openFileName = '전자담배 증상_944_naver_blog.txt'
    
f = open(openFileName, 'r', encoding='utf-8')
data = f.read()
f.close()

In [9]:
data=data.replace('\n','')
for i in range(1,944):
    x="naver_blog_"+str(i)
    data=data.replace(x,"\n\n"+x+"  "+"\n")
f = open('전자담배증상_블로그_크롤링3', 'w', encoding='utf-8')
f.write(data)
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
openFileName = '전자담배증상_블로그_크롤링3'
    
f = open(openFileName, 'r', encoding='utf-8')
result = f.read()
f.close()

In [11]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(result)

In [12]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가가': 0, '가게': 1, '가격': 2, '가결': 3, '가경': 4, '가경동': 5, '가계비': 6, '가공': 7, '가공비': 8, '가공품': 9, '가관': 10, '가구': 11, '가권': 12, '가글': 13, '가글링': 14, '가기': 15, '가까이': 16, '가끔': 17, '가나': 18, '가나다': 19, '가내': 20, '가늠': 21, '가능': 22, '가능성': 23, '가다가': 24, '가닥': 25, '가도': 26, '가동': 27, '가득': 28, '가든': 29, '가디건': 30, '가디언': 31, '가뜩이나': 32, '가라': 33, '가락동': 34, '가락시장': 35, '가래': 36, '가래떡': 37, '가랬는데': 38, '가량': 39, '가레': 40, '가렛': 41, '가려움': 42, '가려움증': 43, '가렵': 44, '가렵거': 45, '가렵기': 46, '가령': 47, '가로': 48, '가로수': 49, '가로수길': 50, '가로수로': 51, '가루': 52, '가르마': 53, '가르킨': 54, '가마': 55, '가마솥': 56, '가망': 57, '가맹': 58, '가면': 59, '가명': 60, '가모': 61, '가무': 62, '가문': 63, '가물': 64, '가뭄': 65, '가미': 66, '가방': 67, '가배': 68, '가벼웁': 69, '가변': 70, '가변저항': 71, '가비': 72, '가쁘': 73, '가쁜': 74, '가사': 75, '가산': 76, '가상현실': 77, '가성': 78, '가세': 79, '가속': 80, '가수': 81, '가쉽': 82, '가스': 83, '가슴': 84, '가슴속': 85, '가슴통': 86, '가습기': 87, '가시': 88, '가시와': 89, '가식': 90, '가신': 91, '가안': 92, '가압류': 93, '가액': 94, '가야': 95, '가양': 9

In [13]:
corpus_path = '전자담배증상_블로그_크롤링3'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [14]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[]
[(262, 1), (285, 1), (574, 1), (609, 1), (1212, 1), (1329, 1), (2274, 1), (2293, 1), (2611, 1), (2639, 1), (3076, 1), (3649, 1), (3929, 1), (4147, 1), (4454, 2), (4519, 1), (4587, 1), (4744, 1), (6313, 1), (6587, 2), (7595, 1), (8171, 1), (8985, 1), (9067, 2), (9560, 1), (10152, 1), (11196, 1), (11444, 1), (12276, 1), (12280, 1), (12292, 2), (12765, 1), (13594, 7), (13662, 2), (13686, 2)]
[]
[]
[(292, 1), (464, 1), (824, 5), (1107, 1), (1329, 2), (1579, 1), (1957, 1), (1958, 1), (2597, 3), (2707, 1), (3714, 1), (3739, 1), (3966, 1), (4438, 3), (4519, 1), (5870, 1), (5908, 1), (6867, 1), (7502, 1), (7725, 1), (7734, 1), (7774, 1), (7969, 1), (7971, 2), (8211, 5), (8738, 1), (8953, 1), (9532, 1), (10125, 5), (10402, 1), (10776, 3), (11782, 1), (11799, 1), (11833, 1), (12984, 1), (13043, 6), (13558, 1), (13720, 1)]
[]
[]
[(119, 1), (665, 1), (1322, 1), (2274, 1), (2294, 1), (2532, 2), (2611, 1), (3127, 1), (3805, 1), (4203, 1), (4209, 1), (4438, 1), (4883, 3), (6274, 1), (7969, 2

[(133, 1), (1027, 1), (1431, 5), (1450, 13), (2127, 2), (2272, 1), (2293, 1), (2407, 2), (2470, 1), (2611, 1), (2614, 1), (2861, 1), (3528, 3), (3533, 1), (3681, 2), (3685, 1), (3747, 1), (3966, 2), (4560, 1), (4587, 2), (5677, 2), (5766, 1), (5815, 1), (6125, 3), (6614, 1), (6998, 1), (7098, 1), (7119, 1), (7209, 1), (7299, 6), (7971, 1), (8117, 1), (8409, 1), (8738, 2), (8902, 1), (9431, 1), (10287, 1), (10288, 1), (10573, 1), (10760, 1), (10799, 1), (10806, 1), (11341, 1), (11422, 1), (12616, 1), (12638, 2), (12883, 1), (12922, 2), (13010, 1), (13450, 1), (13460, 4), (13475, 1), (13690, 1), (13862, 3)]
[]
[]
[(1486, 1), (1487, 1), (2420, 1), (3630, 1), (6601, 1), (8204, 1)]
[]
[]
[(119, 1), (133, 2), (216, 1), (285, 2), (594, 1), (665, 1), (863, 1), (1202, 3), (1322, 2), (1471, 1), (1480, 1), (1573, 1), (1930, 1), (1962, 1), (2085, 1), (2141, 1), (2274, 2), (2289, 1), (2291, 1), (2293, 2), (2294, 3), (2546, 1), (2611, 2), (2614, 2), (2882, 1), (3076, 1), (3098, 2), (3127, 1), (3577,

[]
[]
[(17, 1), (285, 1), (500, 1), (609, 1), (709, 1), (1032, 1), (1873, 2), (2392, 1), (4306, 1), (4438, 1), (7597, 1), (9587, 1), (12491, 1), (13746, 1)]
[]
[]
[(2, 1), (133, 2), (603, 1), (794, 1), (1022, 2), (1183, 6), (1317, 1), (1482, 1), (1492, 2), (1510, 1), (1956, 1), (2152, 4), (2238, 2), (2465, 1), (2493, 1), (2611, 3), (2946, 2), (4835, 1), (4841, 1), (5353, 1), (5727, 4), (7181, 1), (7734, 6), (8642, 1), (9000, 1), (9175, 1), (9301, 4), (9594, 1), (10109, 1), (10125, 1), (10370, 1), (10423, 4), (10487, 1), (10639, 1), (10640, 1), (10975, 1), (11183, 1), (11503, 1), (12463, 1), (13046, 3), (13431, 1), (13659, 1), (13674, 4)]
[]
[]
[(464, 1), (986, 1), (1337, 1), (3086, 1), (3751, 1), (3866, 1), (6161, 1), (6340, 1), (7033, 1), (8764, 1), (8884, 1), (10125, 1), (10672, 1), (11300, 1), (12441, 1), (12673, 1), (13115, 3), (13137, 1), (13521, 1)]
[]
[]
[(36, 1), (1450, 2), (2421, 1), (2611, 2), (2722, 1), (3969, 1), (5815, 1), (6253, 1), (8553, 1), (8738, 1), (8953, 1), (9299,

[]
[(2, 1), (119, 1), (170, 1), (824, 1), (1022, 3), (1492, 5), (2007, 1), (2274, 1), (2291, 1), (2400, 1), (2611, 3), (2772, 1), (2891, 1), (3305, 1), (4587, 1), (5752, 1), (5758, 1), (6263, 1), (6587, 3), (6601, 1), (7622, 1), (8503, 2), (8522, 1), (9506, 1), (9654, 1), (10125, 2), (10152, 1), (10370, 2), (10919, 1), (11083, 1), (11358, 1), (12280, 1)]
[]
[]
[(107, 1), (293, 1), (1736, 3), (2489, 1), (2614, 1), (4726, 1), (5869, 1), (6136, 1), (7411, 1), (9346, 1), (10125, 1), (10919, 1), (12027, 1), (12280, 1), (13245, 1), (13411, 1), (13873, 1)]
[]
[]
[(119, 1), (2293, 1), (2532, 1), (4438, 1), (6232, 1), (6601, 2), (7969, 1), (8409, 1), (8747, 1), (9779, 3), (10125, 5), (10269, 1), (10370, 2), (13115, 1)]
[]
[]
[(213, 1), (493, 1), (2115, 1), (2274, 1), (3714, 1), (4438, 1), (6233, 1), (8185, 1), (8322, 1), (10096, 1), (10269, 2), (11169, 1), (11415, 1), (12466, 1), (13115, 2)]
[]
[]
[(22, 1), (170, 1), (1397, 1), (1450, 1), (1612, 1), (2238, 1), (4641, 1), (7734, 1), (8204, 1), (

[(119, 2), (980, 2), (1146, 1), (1432, 1), (1944, 1), (2238, 1), (2420, 5), (2766, 1), (2852, 1), (3001, 1), (3539, 1), (3805, 1), (4438, 1), (4519, 1), (4797, 1), (5994, 1), (6020, 1), (6253, 1), (8264, 8), (9830, 2), (10125, 1), (10527, 1), (11444, 1), (11521, 1), (12280, 2), (13029, 1)]
[]
[]
[(1329, 2), (1492, 1), (2611, 1), (3127, 1), (3513, 1), (5727, 1), (6601, 1), (7734, 1), (8118, 1), (9752, 1), (9779, 1), (10125, 2), (10269, 1), (10554, 1), (10799, 1), (11203, 1), (13647, 1)]
[]
[]
[(2546, 1), (2639, 1), (5352, 1), (9216, 1), (10213, 1), (10217, 1), (12918, 1)]
[]
[]
[(17, 1), (119, 1), (216, 1), (464, 2), (544, 1), (709, 2), (887, 1), (1047, 1), (1308, 1), (1325, 1), (1329, 2), (1337, 1), (1405, 1), (1455, 1), (1776, 1), (2274, 3), (2532, 2), (2611, 1), (2614, 1), (2933, 1), (3649, 1), (3749, 1), (3805, 2), (3946, 1), (3966, 1), (4167, 3), (4438, 1), (4867, 1), (4938, 1), (5441, 1), (6281, 1), (6446, 2), (6601, 1), (6989, 1), (7432, 2), (7502, 3), (7629, 1), (7734, 3), (7932

In [ ]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=13929, num_topics=10, decay=0.5, chunksize=2000)


In [ ]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [ ]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 13929)
1.0000001


In [ ]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
#pyLDAvis.display(prepared_data)
pyLDAvis.show(prepared_data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8909/    [Ctrl-C to exit]


127.0.0.1 - - [21/May/2019 20:18:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2019 20:18:38] "GET /LDAvis.css HTTP/1.1" 200 -
